In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [160]:
df_otodom = pd.read_csv('datasets/otodom_dataset_raw.csv')

In [127]:
df_otodom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   url                   6820 non-null   object
 1   name/title            6820 non-null   object
 2   address               6820 non-null   object
 3   price                 6820 non-null   object
 4   area                  6820 non-null   object
 5   price-per-area        6526 non-null   object
 6   floor/store           6720 non-null   object
 7   no of rooms           6820 non-null   object
 8   year of construction  6154 non-null   object
 9   parking space         4206 non-null   object
 10  market                6154 non-null   object
 11  form of ownership     4941 non-null   object
 12  condition             5229 non-null   object
 13  rent                  2846 non-null   object
 14  heating               4913 non-null   object
 15  advertiser type       6154 non-null   

In [128]:
df_otodom.iloc[13:30, 3:4]

,price
13,1 100 000 zł
14,Zapytaj o cenę
15,835 000 zł
16,925 443 zł
17,1 027 752 zł
18,Zapytaj o cenę
19,630 000 zł
20,1 080 000 zł
21,731 157 zł
22,1 500 000 zł


In [19]:
df_otodom.head()

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
0,https://www.otodom.pl/pl/oferta/gotowe-2-pokoj...,Gotowe| 2 pokoje| blisko centrum| Bonarka,"Wola Duchacka, Podgórze Duchackie, Kraków, mał...",719 000 zł,"55,98 m²",12 844 zł/m²,parter/8,3,2022,NaN,pierwotny,NaN,do wykończenia,NaN,miejskie,biuro nieruchomości,tak,ogródek,apartamentowiec,brak informacji
1,https://www.otodom.pl/pl/oferta/4-pok-mieszkan...,4-pok.mieszkanie z Sauną - Wysoki Standard ! 2...,"Biskupa Albina Małysiaka, Kobierzyn, Dębniki, ...",1 350 000 zł,87 m²,15 517 zł/m²,2/4,4,2021,garaż/miejsce parkingowe,wtórny,pełna własność,NaN,900 zł,miejskie,biuro nieruchomości,tak,balkon,brak informacji,cegła
2,https://www.otodom.pl/pl/oferta/3-pokoje-w-rza...,3 pokoje w rządowym programie kredyt 2%,"Bieżanów, Bieżanów-Prokocim, Kraków, małopolskie",599 000 zł,"47,1 m²",12 718 zł/m²,1/4,3,2023,garaż/miejsce parkingowe,pierwotny,pełna własność,do wykończenia,NaN,miejskie,biuro nieruchomości,tak,balkon,blok,brak informacji
3,https://www.otodom.pl/pl/oferta/mieszkanie-ide...,"Mieszkanie idealne na start, 3 - pokoje!","ul. Erazma Jerzmanowskiego, Prokocim, Bieżanów...",833 500 zł,"66,69 m²",12 498 zł/m²,3/8,3,2024,garaż/miejsce parkingowe,pierwotny,NaN,NaN,NaN,miejskie,biuro nieruchomości,tak,balkon,apartamentowiec,brak informacji
4,https://www.otodom.pl/pl/oferta/ul-lasowka-3-p...,"ul. Lasówka, 3 pokoje, 65m2 + taras 20m2!","ul. Lasówka, Płaszów, Podgórze, Kraków, małopo...",949 000 zł,"65,81 m²",14 420 zł/m²,7/7,3,2023,NaN,wtórny,NaN,NaN,NaN,NaN,biuro nieruchomości,nie,NaN,brak informacji,brak informacji


In [129]:
df_otodom.describe()

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
count,6820,6820,6820,6820,6820,6526,6720,6820,6154,4206,6154,4941,5229,2846,4913,6154,6154,5088,6154,6154
unique,6820,6177,1161,2351,2344,3235,126,11,147,1,2,4,3,351,5,3,2,10,8,10
top,https://www.otodom.pl/pl/oferta/gotowe-2-pokoj...,Limitowane Okazje na Dzień Otwarty! Bezpośrednio,"Stare Podgórze, Podgórze, Kraków, małopolskie",Zapytaj o cenę,"38,17 m²",12 500 zł/m²,3/4,2,2023,garaż/miejsce parkingowe,wtórny,pełna własność,do wykończenia,500 zł,miejskie,biuro nieruchomości,tak,balkon,blok,brak informacji
freq,1,26,278,294,62,69,382,2448,1075,4206,3719,4770,2708,281,3776,5454,3328,3682,2711,3447


In [176]:
#7 wierszy ma cenę w EUR
df_otodom[df_otodom['price'].str.contains(r'\d', na=False) & ~df_otodom['price'].str.contains('zł', na=False)].count()

url                     7
name/title              7
address                 7
price                   7
area                    7
price-per-area          7
floor/store             7
no of rooms             7
year of construction    7
parking space           2
market                  7
form of ownership       6
condition               6
rent                    4
heating                 5
advertiser type         7
elevator                7
outdoor area            5
building type           7
building material       7
dtype: int64

In [146]:
df_otodom['price'] = df_otodom['price'].str.replace(' ', '')
df_otodom['price'] = df_otodom['price'].str.extract('(\d+)').astype(float)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\katar\AppData\Local\Temp\ipykernel_15824\2660707632.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_otodom['price'] = df_otodom['price'].str.extract('(\d+)').astype(float)


In [147]:
df_otodom['area'] = df_otodom['area'].str.replace(' m²', '')
df_otodom['area'] = df_otodom['area'].str.replace(',', '.').astype(float)

In [172]:
df_otodom[df_otodom['price-per-area'].str.contains(r'\d', na=False) & ~df_otodom['price-per-area'].str.contains('zł', na=False)]

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material


In [148]:
df_otodom['price-per-area'] = df_otodom['price-per-area'].str.replace(' zł/m²', '').str.replace(' ', '')
df_otodom['price-per-area'] = df_otodom['price-per-area'].astype(float)

In [173]:
#trzy wiersze w innej walucie
df_otodom[df_otodom['rent'].str.contains(r'\d', na=False) & ~df_otodom['rent'].str.contains('zł', na=False)]

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
3977,https://www.otodom.pl/pl/oferta/chorwacja-spli...,Chorwacja Split luksusowy apartament,"Krowodrza, Krowodrza, Kraków, małopolskie",585 280 EUR,"91,45 m²",27 535 zł/m²,3/4,4,2025,garaż/miejsce parkingowe,pierwotny,pełna własność,do zamieszkania,100 EUR,elektryczne,prywatny,tak,"balkon, taras",apartamentowiec,silikat
4908,https://www.otodom.pl/pl/oferta/penthouse-w-od...,Penthouse w odrestaurowanej kamienicy - taras ...,"ul. marsz. Józefa Piłsudskiego, Piasek, Stare ...",1 150 000 EUR,228 m²,21 700 zł/m²,4/5,5,1906,NaN,wtórny,pełna własność,do zamieszkania,220 EUR,gazowe,biuro nieruchomości,tak,"balkon, taras",kamienica,cegła
6335,https://www.otodom.pl/pl/oferta/2m-wysoki-stan...,2M! Wysoki Standard| Gotowe| Balkon | Widok!,"Wola Duchacka, Podgórze Duchackie, Kraków, mał...",620 000 zł,"35,55 m²",17 440 zł/m²,7/11,2,2019,NaN,wtórny,pełna własność,do zamieszkania,420 GBP,miejskie,biuro nieruchomości,tak,balkon,blok,brak informacji


In [149]:
df_otodom['rent'] = df_otodom['rent'].str.extract('(\d+)').astype(float)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\katar\AppData\Local\Temp\ipykernel_15824\3216866654.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_otodom['rent'] = df_otodom['rent'].str.extract('(\d+)').astype(float)


In [157]:
df_otodom['no of rooms'] = df_otodom['no of rooms'].str.replace(' ', '')
df_otodom['no of rooms'] = df_otodom['no of rooms'].str.extract('(\d+)').astype(int)


<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\katar\AppData\Local\Temp\ipykernel_15824\528362110.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_otodom['no of rooms'] = df_otodom['no of rooms'].str.extract('(\d+)').astype(int)


In [158]:
df_otodom.head()

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
0,https://www.otodom.pl/pl/oferta/gotowe-2-pokoj...,Gotowe| 2 pokoje| blisko centrum| Bonarka,"Wola Duchacka, Podgórze Duchackie, Kraków, mał...",719000.0,55.98,12844.0,parter/8,3,2022,NaN,pierwotny,NaN,do wykończenia,NaN,miejskie,biuro nieruchomości,tak,ogródek,apartamentowiec,brak informacji
1,https://www.otodom.pl/pl/oferta/4-pok-mieszkan...,4-pok.mieszkanie z Sauną - Wysoki Standard ! 2...,"Biskupa Albina Małysiaka, Kobierzyn, Dębniki, ...",1350000.0,87.00,15517.0,2/4,4,2021,garaż/miejsce parkingowe,wtórny,pełna własność,NaN,900.0,miejskie,biuro nieruchomości,tak,balkon,brak informacji,cegła
2,https://www.otodom.pl/pl/oferta/3-pokoje-w-rza...,3 pokoje w rządowym programie kredyt 2%,"Bieżanów, Bieżanów-Prokocim, Kraków, małopolskie",599000.0,47.10,12718.0,1/4,3,2023,garaż/miejsce parkingowe,pierwotny,pełna własność,do wykończenia,NaN,miejskie,biuro nieruchomości,tak,balkon,blok,brak informacji
3,https://www.otodom.pl/pl/oferta/mieszkanie-ide...,"Mieszkanie idealne na start, 3 - pokoje!","ul. Erazma Jerzmanowskiego, Prokocim, Bieżanów...",833500.0,66.69,12498.0,3/8,3,2024,garaż/miejsce parkingowe,pierwotny,NaN,NaN,NaN,miejskie,biuro nieruchomości,tak,balkon,apartamentowiec,brak informacji
4,https://www.otodom.pl/pl/oferta/ul-lasowka-3-p...,"ul. Lasówka, 3 pokoje, 65m2 + taras 20m2!","ul. Lasówka, Płaszów, Podgórze, Kraków, małopo...",949000.0,65.81,14420.0,7/7,3,2023,NaN,wtórny,NaN,NaN,NaN,NaN,biuro nieruchomości,nie,NaN,brak informacji,brak informacji


In [151]:
df_otodom['price'].mean()

984641.4043824701

In [159]:
df_otodom.describe()

,price,area,price-per-area,no of rooms,rent
count,6.526000e+03,6820.000000,6526.000000,6820.000000,2846.000000
mean,9.846414e+05,60.393113,16571.950506,2.717155,478.795854
std,6.596879e+05,34.785674,4544.521552,1.066453,233.963362
min,9.500000e+04,10.270000,271.000000,1.000000,0.000000
25%,6.650000e+05,41.397500,13711.000000,2.000000,350.000000
50%,8.340000e+05,53.500000,16000.000000,3.000000,500.000000
75%,1.081038e+06,68.862500,18500.000000,3.000000,650.000000
max,1.350000e+07,1000.000000,70000.000000,10.000000,980.000000


In [153]:
df_nieruchomosci = pd.read_csv('datasets/nieruchomosci-online_dataset_raw.csv')

In [154]:
df_nieruchomosci.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3949 entries, 0 to 3948
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   url                                  3949 non-null   object 
 1   name/title                           3949 non-null   object 
 2   address                              3848 non-null   object 
 3   price                                3920 non-null   object 
 4   area                                 3949 non-null   object 
 5   price-per-area                       3920 non-null   object 
 6   floor/store                          3949 non-null   object 
 7   no of floors/stores in the building  3710 non-null   float64
 8   no of rooms                          3949 non-null   object 
 9   year of construction                 3949 non-null   object 
 10  parking space                        3949 non-null   object 
 11  market                        

In [177]:
df_nieruchomosci.head()

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
0,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żelechowskiego","Żelechowskiego, Bronowice, Kraków, małopolskie",899 000 zł,"51,70 m²","17 388,78 zł/m²",1,3.0,2,1980,tak,wtórny,własność
1,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Balicka","Balicka, Bronowice, Kraków, małopolskie",1 575 000 zł,125 m²,12 600 zł/m²,3,4.0,6,2004,w garażu podziemnym,wtórny,"własność, księga wieczysta"
2,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Zauchy","Zauchy, Górka Narodowa, Kraków, małopolskie",1 250 000 zł,"65,23 m²","19 162,96 zł/m²",2,5.0,3,2023,tak,wtórny,NaN
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Racławicka","Racławicka, Krowodrza, Kraków, małopolskie",740 250 zł,"49,35 m²",15 000 zł/m²,6,10.0,2,1950,parking publiczny / na ulicy,wtórny,NaN
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Pustynna","Pustynna, Kliny, Kraków, małopolskie",1 200 000 zł,"71,06 m²","16 887,14 zł/m²",parter,1.0,3,2013,garaż w bryle budynku,wtórny,"własność, księga wieczysta"


In [179]:
#9 cen jest w innej walucie
df_nieruchomosci[df_nieruchomosci['price'].str.contains(r'\d', na=False) & ~df_nieruchomosci['price'].str.contains('zł', na=False)].count()

url                                    9
name/title                             9
address                                9
price                                  9
area                                   9
price-per-area                         9
floor/store                            9
no of floors/stores in the building    9
no of rooms                            9
year of construction                   9
parking space                          9
market                                 9
form of ownership                      1
dtype: int64

In [180]:
df_nieruchomosci['price'] = df_nieruchomosci['price'].str.replace(' ', '')
df_nieruchomosci['price'] = df_nieruchomosci['price'].str.extract('(\d+)').astype(float)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\katar\AppData\Local\Temp\ipykernel_15824\1641507478.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_nieruchomosci['price'] = df_nieruchomosci['price'].str.extract('(\d+)').astype(float)


In [182]:
df_nieruchomosci['area'] = df_nieruchomosci['area'].str.replace(' m²', '')
df_nieruchomosci['area'] = df_nieruchomosci['area'].str.replace(',', '.').astype(float)

In [185]:
#9 cen jest w innej walucie
df_nieruchomosci[df_nieruchomosci['price-per-area'].str.contains(r'\d', na=False) & ~df_nieruchomosci['price-per-area'].str.contains('zł', na=False)].count()

url                                    9
name/title                             9
address                                9
price                                  9
area                                   9
price-per-area                         9
floor/store                            9
no of floors/stores in the building    9
no of rooms                            9
year of construction                   9
parking space                          9
market                                 9
form of ownership                      1
dtype: int64

In [187]:
df_nieruchomosci['price-per-area'] = df_nieruchomosci['price-per-area'].str.replace(' ', '')
df_nieruchomosci['price-per-area'] = df_nieruchomosci['price-per-area'].str.extract('(\d+)').astype(float)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\katar\AppData\Local\Temp\ipykernel_15824\1416773852.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_nieruchomosci['price-per-area'] = df_nieruchomosci['price-per-area'].str.extract('(\d+)').astype(float)


In [201]:
#niektóre mieszkania zamiast liczby pokoi mają wpisane "parter/suterena/-"?
df_nieruchomosci[~df_nieruchomosci['no of rooms'].str.contains(r'\d')]

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
37,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Wieczysta, Aleja Jana Pawła II 52",NaN,1373420.0,79.85,17200.0,II kwartał 2024,NaN,parter,4,-,II kwartał 2024,pierwotny (oferta dewelopera)
45,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Gotyk, ul. Grabczaka 8",NaN,NaN,58.36,NaN,III kwartał 2024,NaN,suterena,3,garaż,dostępne,NaN
71,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Bronowice Wielkie, ul. Piaskow...",NaN,903189.0,73.43,12300.0,oddana do użytku,NaN,parter,3,naziemne,"IV kwartał 2022, oddana do użytku",pierwotny (oferta dewelopera)
119,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Wola Justowska, ul. Królowej J...",NaN,1221192.0,72.69,16800.0,zrealizowana,NaN,parter,3,garaż,"październik 2023, zrealizowana",pierwotny (oferta dewelopera)
178,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Tonie, ul. Władysława Łokietka",NaN,1638000.0,136.60,11991.0,październik 2025,NaN,parter,4,garaż,październik 2025,pierwotny (oferta dewelopera)
212,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Prądnik Biały, Piaszczysta/Pęk...",NaN,917868.0,62.44,14700.0,IV kwartał 2024,NaN,parter,3,garaż,IV kwartał 2024,pierwotny (oferta dewelopera)
221,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Olszyny, ul. Petrażyckiego",NaN,NaN,55.00,NaN,wrzesień 2024,NaN,parter,3,naziemne,zarezerwowane,NaN
266,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Krowodrza Górka, ul. Józefa Wy...",NaN,1130000.0,72.34,15620.0,I kwartał 2024,NaN,parter,3,garaż,I kwartał 2024,pierwotny (oferta dewelopera)
268,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Krowodrza Górka, ul. Józefa Wy...",NaN,680000.0,38.88,17489.0,I kwartał 2024,NaN,parter,2,garaż,I kwartał 2024,pierwotny (oferta dewelopera)
274,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Dziekanowice, ul. Piasta Kołod...",NaN,NaN,45.01,NaN,III kwartał 2025,NaN,parter,3,garaż,dostępne,NaN


In [190]:
#df_nieruchomosci['no of rooms'] = df_nieruchomosci['no of rooms'].str.replace(' ', '')
#df_nieruchomosci['no of rooms'] = df_nieruchomosci['no of rooms'].str.extract('(\d+)').astype(int)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\katar\AppData\Local\Temp\ipykernel_15824\4183817636.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_nieruchomosci['no of rooms'] = df_nieruchomosci['no of rooms'].str.extract('(\d+)').astype(int)
C:\Users\katar\AppData\Local\Temp\ipykernel_15824\4183817636.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_nieruchomosci['no of rooms'] = df_nieruchomosci['no of rooms'].str.extract('(\d+)').astype(int)


ValueError: cannot convert float NaN to integer

In [202]:
df_nieruchomosci.head()

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
0,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żelechowskiego","Żelechowskiego, Bronowice, Kraków, małopolskie",899000.0,51.70,17388.0,1,3.0,2,1980,tak,wtórny,własność
1,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Balicka","Balicka, Bronowice, Kraków, małopolskie",1575000.0,125.00,12600.0,3,4.0,6,2004,w garażu podziemnym,wtórny,"własność, księga wieczysta"
2,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Zauchy","Zauchy, Górka Narodowa, Kraków, małopolskie",1250000.0,65.23,19162.0,2,5.0,3,2023,tak,wtórny,NaN
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Racławicka","Racławicka, Krowodrza, Kraków, małopolskie",740250.0,49.35,15000.0,6,10.0,2,1950,parking publiczny / na ulicy,wtórny,NaN
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Pustynna","Pustynna, Kliny, Kraków, małopolskie",1200000.0,71.06,16887.0,parter,1.0,3,2013,garaż w bryle budynku,wtórny,"własność, księga wieczysta"


In [203]:
df_nieruchomosci.describe()

,price,area,price-per-area,no of floors/stores in the building
count,3.920000e+03,3949.000000,3920.000000,3710.000000
mean,1.023459e+06,62.012249,16783.809439,4.850135
std,7.258431e+05,36.452915,4992.176616,2.531264
min,1.030000e+05,12.300000,1816.000000,1.000000
25%,6.800000e+05,41.760000,13802.500000,3.000000
50%,8.406720e+05,54.770000,16072.500000,4.000000
75%,1.116449e+06,70.000000,18926.000000,6.000000
max,1.600000e+07,795.000000,71018.000000,45.000000
